# Bước 4: FP-Growth Modeling for Association Rules

Notebook này sử dụng ma trận `basket_bool` (được chuẩn bị ở Bước 02) để:

- Khai thác tập mục phổ biến (frequent itemsets) bằng thuật toán FP-Growth
- Sinh luật kết hợp (association rules) với các chỉ số: `support`, `confidence`, `lift`
- Lọc luật theo các ngưỡng do người dùng cấu hình
- Trực quan hoá một số nhóm luật tiêu biểu phục vụ storytelling & phân tích kinh doanh
- So sánh sơ bộ thời gian chạy và số lượng luật thu được so với Apriori (sẽ chi tiết hơn ở Bước 5)

Notebook được thiết kế theo kiểu parameterized để dễ dàng tích hợp với papermill.


In [ ]:
# Parameters injected by papermill

# Đường dẫn tới basket_bool được tạo từ Notebook 02
BASKET_BOOL_PATH = "data/processed/basket_bool.parquet"

# Đường dẫn lưu file luật kết hợp sau khi lọc (FP-Growth)
RULES_OUTPUT_PATH = "data/processed/rules_fpgrowth_filtered.csv"

# Tham số cho bước khai thác tập mục phổ biến (frequent itemsets)
MIN_SUPPORT = 0.01  # ngưỡng support tối thiểu
MAX_LEN = 3         # độ dài tối đa của itemset (số sản phẩm trong 1 tập)

# Tham số cho bước sinh luật
METRIC = "lift"     # chỉ số dùng để generate rules: 'support' / 'confidence' / 'lift'
MIN_THRESHOLD = 1.0 # ngưỡng tối thiểu cho METRIC

# Tham số lọc luật sau khi generate
FILTER_MIN_SUPPORT = 0.01
FILTER_MIN_CONF = 0.3
FILTER_MIN_LIFT = 1.2
FILTER_MAX_ANTECEDENTS = 2
FILTER_MAX_CONSEQUENTS = 1

# Số lượng luật top để vẽ biểu đồ
TOP_N_RULES = 20

# Bật/tắt các biểu đồ matplotlib
PLOT_TOP_LIFT = True
PLOT_TOP_CONF = True
PLOT_SCATTER = True
PLOT_NETWORK = True

# Bật/tắt biểu đồ HTML tương tác (Plotly)
PLOT_PLOTLY_SCATTER = True


## Set up

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

# Biểu đồ tương tác HTML
import plotly.express as px

# Determine correct project root
cwd = os.getcwd()
if os.path.basename(cwd) == "notebooks":
    project_root = os.path.abspath("..")
else:
    project_root = cwd

src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

from cluster_library import FPGrowthMiner, DataVisualizer  


## Thiết lập style vẽ biểu đồ


In [ ]:
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.titlesize"] = 14
plt.rcParams["axes.labelsize"] = 12

visualizer = DataVisualizer()


## Tải basket_bool


In [ ]:
basket_bool = pd.read_parquet(BASKET_BOOL_PATH)

print("=== Thông tin basket_bool ===")
print(f"- Số hoá đơn (rows): {basket_bool.shape[0]:,}")
print(f"- Số sản phẩm (columns): {basket_bool.shape[1]:,}")
print(f"- Tỷ lệ ô = 1 (có mua): {basket_bool.values.mean():.4f}")

basket_bool.head()


## Khai thác tập phổ biến bằng thuật toán FP-Growth


In [ ]:
# Khởi tạo FP-Growth miner
fp_miner = FPGrowthMiner(basket_bool=basket_bool)

start_time = time.time()
frequent_itemsets_fp = fp_miner.mine_frequent_itemsets(
    min_support=MIN_SUPPORT,
    max_len=MAX_LEN,
    use_colnames=True,
)
elapsed_time = time.time() - start_time

print("=== Kết quả khai thác tập mục phổ biến (FP-Growth) ===")
print(f"- Thời gian chạy: {elapsed_time:.2f} giây")
print(f"- Số tập mục phổ biến thu được: {len(frequent_itemsets_fp):,}")

frequent_itemsets_fp.head(10)


In [ ]:
if frequent_itemsets_fp is not None and not frequent_itemsets_fp.empty:
    # Phân phối độ dài các tập mục (1-itemset, 2-itemset, 3-itemset, ...)
    visualizer.plot_itemset_length_distribution(
        frequent_itemsets=frequent_itemsets_fp
    )

    # Top các tập mục phổ biến nhất theo support (ưu tiên itemset có từ 2 sản phẩm trở lên)
    visualizer.plot_top_frequent_itemsets(
        frequent_itemsets=frequent_itemsets_fp,
        top_n=20,
        min_len=2,
    )
else:
    print("Không có frequent itemsets để trực quan hoá.")


## Sinh luật kết hợp từ tập mục phổ biến (FP-Growth)


In [ ]:
rules_fp = fp_miner.generate_rules(
    metric=METRIC,
    min_threshold=MIN_THRESHOLD,
)

# Thêm cột dạng chuỗi dễ đọc
rules_fp = fp_miner.add_readable_rule_str()

print("=== Một vài luật kết hợp đầu tiên (FP-Growth, chưa lọc) ===")
cols_preview = [
    "antecedents_str",
    "consequents_str",
    "support",
    "confidence",
    "lift",
]
rules_fp[cols_preview].head(10)


## Lọc các luật FP-Growth theo ngưỡng support / confidence / lift


In [ ]:
rules_filtered_fp = fp_miner.filter_rules(
    min_support=FILTER_MIN_SUPPORT,
    min_confidence=FILTER_MIN_CONF,
    min_lift=FILTER_MIN_LIFT,
    max_len_antecedents=FILTER_MAX_ANTECEDENTS,
    max_len_consequents=FILTER_MAX_CONSEQUENTS,
)

print("=== Thống kê sau khi lọc luật (FP-Growth) ===")
print(f"- Tổng số luật ban đầu: {rules_fp.shape[0]:,}")
print(f"- Số luật sau khi lọc: {rules_filtered_fp.shape[0]:,}")

rules_filtered_fp[cols_preview].head(10)


In [ ]:
# Cell 16: Top theo lift
if PLOT_TOP_LIFT and not rules_filtered_fp.empty:
    visualizer.plot_top_rules_lift(
        rules_df=rules_filtered_fp,
        top_n=TOP_N_RULES,
    )
else:
    if rules_filtered_fp.empty:
        print("Không có luật nào sau khi lọc để vẽ top lift.")
    else:
        print("PLOT_TOP_LIFT = False, bỏ qua biểu đồ top lift.")


In [ ]:
# Cell 17: Top theo confidence
if PLOT_TOP_CONF and not rules_filtered_fp.empty:
    visualizer.plot_top_rules_confidence(
        rules_df=rules_filtered_fp,
        top_n=TOP_N_RULES,
    )
else:
    if rules_filtered_fp.empty:
        print("Không có luật nào sau khi lọc để vẽ top confidence.")
    else:
        print("PLOT_TOP_CONF = False, bỏ qua biểu đồ top confidence.")


In [ ]:
# Cell 18: Scatter support–confidence
if PLOT_SCATTER and not rules_filtered_fp.empty:
    visualizer.plot_rules_support_confidence_scatter(
        rules_df=rules_filtered_fp,
    )
else:
    if rules_filtered_fp.empty:
        print("Không có luật nào sau khi lọc để vẽ scatter.")
    else:
        print("PLOT_SCATTER = False, bỏ qua biểu đồ scatter.")


In [ ]:
# Cell 19: Scatter Plotly
if PLOT_PLOTLY_SCATTER and not rules_filtered_fp.empty:
    visualizer.plot_rules_support_confidence_scatter_interactive(
        rules_df=rules_filtered_fp,
    )
else:
    if rules_filtered_fp.empty:
        print("Không có luật nào sau khi lọc để vẽ scatter Plotly.")
    else:
        print("PLOT_PLOTLY_SCATTER = False, bỏ qua biểu đồ Plotly.")


In [ ]:
# Cell 20: Network graph
if PLOT_NETWORK and not rules_filtered_fp.empty:
    visualizer.plot_rules_network(
        rules_df=rules_filtered_fp,
        max_rules=min(TOP_N_RULES, 30),
    )
else:
    if rules_filtered_fp.empty:
        print("Không có luật nào sau khi lọc để vẽ network graph.")
    else:
        print("PLOT_NETWORK = False, bỏ qua network graph.")


In [ ]:
# Sort & lấy Top-K rules
rules_topK = (
    rules_filtered_fp
    .sort_values(METRIC, ascending=False)
    .head(TOP_N_RULES)
)

# File dùng cho clustering (full rules)
rules_filtered_fp.to_csv(RULES_OUTPUT_PATH, index=False)

# File dùng cho report / slide (Top-K)
topk_path = RULES_OUTPUT_PATH.replace(".csv", f"_top{TOP_N_RULES}.csv")
rules_topK.to_csv(topk_path, index=False)

print(f"✅ Saved full rules to: {RULES_OUTPUT_PATH}")
print(f"✅ Saved top-{TOP_N_RULES} rules to: {topk_path}")


In [ ]:
# Chọn 10 luật tiêu biểu để đưa vào report
rules_sample_10 = rules_topK[
    ["antecedents", "consequents", "support", "confidence", "lift"]
].head(10)

rules_sample_10
